### Filtrando e organizando datagrames de paradas de onibus

A primeira parte do projeto consiste em criar funções para leituras de arquivos shapefiles

In [1]:
# Abre o dataframe corretamente
import geopandas as gpd

FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"

def abreSHP(caminhoSHP):

    dados_shapefile = gpd.read_file(caminhoSHP)

    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)
num_linha = '0.205'
df_linhas[df_linhas['linha']== num_linha].geometry

140    LINESTRING (-48.11046 -15.79485, -48.11016 -15...
468    LINESTRING (-48.06688 -16.02025, -48.06701 -16...
Name: geometry, dtype: geometry

In [3]:

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)
num_linha = '0.205'
df_linhas[df_linhas['linha']== num_linha].geometry


# Filtrar a linha de ônibus '0.813' do DataFrame df_linhas.
linha_813 = df_linhas[(df_linhas['linha'] == num_linha ) & (df_linhas['sentido'] == 'VOLTA')]
for idx, ponto_linha in linha_813.iterrows():
    print(ponto_linha['geometry'])

LINESTRING (-48.11046148391251 -15.794854892268013, -48.11015506079145 -15.79506100229111, -48.11011561563112 -15.79506186799862, -48.11008284808316 -15.795052409849625, -48.10999186094524 -15.79495602246652, -48.10996182754768 -15.794940889788192, -48.10992870377855 -15.794942661003175, -48.109739121833776 -15.79506525746872, -48.10964263542305 -15.795127917352959, -48.096592971127386 -15.80390226643988, -48.09650814216852 -15.803961351100824, -48.094333600539265 -15.805423987890384, -48.092868365963156 -15.80641706966822, -48.09262682395301 -15.80660921237223, -48.092311541609284 -15.806885106267968, -48.09200195843538 -15.807183569607812, -48.091838058401514 -15.807361474284559, -48.09164599004736 -15.807580957169634, -48.09134651960996 -15.807972027710525, -48.091116100225925 -15.808312345401516, -48.09097952784026 -15.808536711482391, -48.09080432423833 -15.808863435536676, -48.090614654513445 -15.809251969018156, -48.09045757439989 -15.809647495579588, -48.09035399642758 -15.8099

In [27]:
for idx_parada, parada in df_paradas.iterrows():
    print(parada['geometry'].y)

-15.911319542624627
-15.911596643624351
-15.913986437196487
-15.914538892108915
-15.908573383919444
-15.91337094852713
-15.854070733526939
-15.854454085244098
-15.855986562430733
-15.858682708708761
-15.859967244496703
-15.860379961539149
-15.85820752526704
-15.85613144513947
-15.917473000000003
-15.917827999999998
-15.917473000000003
-15.92225271786416
-15.922437000000004
-15.924756960163583
-15.922320160860007
-15.924722824844345
-15.917649110688115
-15.917525325369455
-15.917321412296468
-15.917476064696464
-15.919028375195868
-15.618195238527411
-15.620720084893637
-15.623631858761406
-15.62364758473566
-15.62746170240111
-15.620905611167046
-15.62051729225648
-15.625262812162688
-15.627568394033089
-15.6254363050788
-15.624348676571431
-15.621016183074918
-15.614084989997286
-15.61893031999728
-15.621241682806128
-15.618099148357118
-15.615133748838057
-15.615338611011452
-15.618303349997277
-15.622612739997276
-15.752189278745613
-15.758968793535495
-15.753186571436261
-15.757434

In [10]:

linha_813 = df_linhas[(df_linhas['linha'] == num_linha) & (df_linhas['sentido'] == 'VOLTA')]

In [36]:
import geopandas as gpd
from shapely.geometry import Point
from collections import OrderedDict
import json
import folium

FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"

def abreSHP(caminhoSHP):
    dados_shapefile = gpd.read_file(caminhoSHP)
    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)
num_linha = '0.205'
df_linhas[df_linhas['linha']== num_linha].geometry

linha_813 = df_linhas[(df_linhas['linha'] == num_linha) & (df_linhas['sentido'] == 'VOLTA')]

paradas_na_rota = OrderedDict()

tolerancia = 0.0019
contador = 0

for idx, linha in linha_813.iterrows():
    linha_geometry = linha['geometry']
    if linha_geometry.geom_type == 'LineString':
        for ponto in linha_geometry.coords:
            ponto_x, ponto_y = ponto
            ponto_shapely = Point(ponto_x, ponto_y)
            for idx_parada, parada in df_paradas.iterrows():
                if contador >= 7:
                    break  # Sai do loop se já encontrou 10 paradas
                parada_geometry_x = parada['geometry'].x
                parada_geometry_y = parada['geometry'].y
                distancia = ponto_shapely.distance(Point(parada_geometry_x, parada_geometry_y))
                if distancia < tolerancia:
                    contador += 1
                    print("Encontrei um ponto")
                    paradas_na_rota[contador] = {
                        'latitude': parada_geometry_y,
                        'longitude': parada_geometry_x
                    }

    

output_json_filename = 'paradas_na_rota_em_ordem_teste.json'

with open(output_json_filename, 'w') as json_file:
    json.dump(paradas_na_rota, json_file)

print("Paradas na rota salvas em 'paradas_na_rota_em_ordem_teste.json'.")

mapa = folium.Map(location=[-15.794854892268013, -48.11046148391251], zoom_start=14)  # Coordenadas do Rio de Janeiro como exemplo

# contador_novo = 0
for cont, coords in paradas_na_rota.items():
    latitude, longitude = coords['latitude'], coords['longitude']
    label = cont  # Os índices começam em 0, então somamos 1
    folium.Marker([latitude, longitude], tooltip=label).add_to(mapa)
    # contador_novo += 1
    # if contador_novo == 9:
    #     break

mapa


Encontrei um ponto
Encontrei um ponto
Encontrei um ponto
Encontrei um ponto
Encontrei um ponto
Encontrei um ponto
Encontrei um ponto
Paradas na rota salvas em 'paradas_na_rota_em_ordem_teste.json'.


In [11]:
import geopandas as gpd
from shapely.geometry import Point
import json


FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"


def abreSHP(caminhoSHP):

    dados_shapefile = gpd.read_file(caminhoSHP)

    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)
num_linha = '0.205'
df_linhas[df_linhas['linha']== num_linha].geometry


# Filtrar a linha de ônibus '0.813' do DataFrame df_linhas.
linha_813 = df_linhas[(df_linhas['linha'] == num_linha ) & (df_linhas['sentido'] == 'VOLTA')]

# Crie um dicionário para armazenar as paradas que fazem parte da rota na ordem de comparação.
paradas_na_rota = {}

# Define a tolerância de proximidade em graus decimais.
tolerancia = 0.00019  # Cerca de 10 metros em graus decimais (para EPSG:4326).

# Itere pelos pontos da linha de ônibus.
for idx, ponto_linha in linha_813.iterrows():
    ponto_linha_geometry = ponto_linha['geometry']
    # Itere pelas paradas.
    for idx_parada, parada in df_paradas.iterrows():
        parada_geometry = parada['geometry']
        # Calcule a distância entre o ponto da linha de ônibus e a parada em graus decimais.
        distancia = ponto_linha_geometry.distance(parada_geometry)
        
        # Se a distância for menor que a tolerância, considere a parada parte da rota.
        if distancia < tolerancia:
            # Armazene a parada no dicionário na ordem em que foram comparadas.
            paradas_na_rota[idx_parada] = {
                'latitude': parada_geometry.y,
                'longitude': parada_geometry.x
            }

# O dicionário 'paradas_na_rota' agora contém as paradas que fazem parte da rota da linha de ônibus '0.813' na ordem de comparação.
# Define o nome do arquivo JSON de saída.
output_json_filename = 'paradas_na_rota_em_ordem.json'

# Salva o dicionário em um arquivo JSON.
with open(output_json_filename, 'w') as json_file:
    json.dump(paradas_na_rota, json_file)

print("Paradas na rota salvas em 'paradas_na_rota_em_ordem.json'.")

Paradas na rota salvas em 'paradas_na_rota_em_ordem.json'.


In [7]:
### Modificado para todas as linhas de onibus

import geopandas as gpd
from shapely.geometry import Point
import json

FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"


def abreSHP(caminhoSHP):
    dados_shapefile = gpd.read_file(caminhoSHP)
    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)

# Crie um dicionário para armazenar as paradas de todas as linhas de ônibus.
todas_as_paradas = {}

# Define a tolerância de proximidade em graus decimais.
tolerancia = 0.00019  # Cerca de 10 metros em graus decimais (para EPSG:4326).

# Itere pelas linhas de ônibus únicas.
for num_linha in df_linhas['linha'].unique():
    # Filtrar a linha de ônibus atual do DataFrame df_linhas.
    linha_atual = df_linhas[(df_linhas['linha'] == num_linha) & (df_linhas['sentido'] == 'CIRCULAR')]

    # Crie um dicionário para armazenar as paradas que fazem parte da rota da linha atual na ordem de comparação.
    paradas_na_rota = {}

    # Itere pelos pontos da linha de ônibus atual.
    for idx, ponto_linha in linha_atual.iterrows():
        ponto_linha_geometry = ponto_linha['geometry']
        # Itere pelas paradas.
        for idx_parada, parada in df_paradas.iterrows():
            parada_geometry = parada['geometry']
            # Calcule a distância entre o ponto da linha de ônibus e a parada em graus decimais.
            distancia = ponto_linha_geometry.distance(parada_geometry)

            # Se a distância for menor que a tolerância, considere a parada parte da rota.
            if distancia < tolerancia:
                # Armazene a parada no dicionário na ordem em que foram comparadas.
                paradas_na_rota[idx_parada] = {
                    'latitude': parada_geometry.y,
                    'longitude': parada_geometry.x
                }

    # Adicione o dicionário de paradas da linha atual ao dicionário geral de todas as paradas.
    todas_as_paradas[num_linha] = paradas_na_rota

# O dicionário 'todas_as_paradas' agora contém as paradas que fazem parte da rota de todas as linhas de ônibus na ordem de comparação.
# Define o nome do arquivo JSON de saída.
output_json_filename = 'paradas_linhas_circular.json'

# Salva o dicionário em um arquivo JSON.
with open(output_json_filename, 'w') as json_file:
    json.dump(todas_as_paradas, json_file)


In [4]:
import geopandas as gpd
from shapely.geometry import Point
import json

FILE_PARADAS = "../../../databases/paradas/paradas.shp"
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"

def abreSHP(caminhoSHP):
    dados_shapefile = gpd.read_file(caminhoSHP)
    return dados_shapefile

df_linhas = abreSHP(FILE_LINHAS)
df_paradas = abreSHP(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)

# Crie um dicionário para armazenar as paradas de todas as linhas de ônibus.
todas_as_paradas = {}

# Define a tolerância de proximidade em graus decimais.
tolerancia = 0.00019  # Cerca de 10 metros em graus decimais (para EPSG:4326).

# Itere pelas linhas de ônibus únicas.
for num_linha in df_linhas['linha'].unique():
    # Filtrar a linha de ônibus atual do DataFrame df_linhas.
    linha_atual = df_linhas[(df_linhas['linha'] == num_linha) & (df_linhas['sentido'] == 'IDA')]

    # Crie um dicionário ordenado para armazenar as paradas na ordem de comparação.
    paradas_ordenadas = {}

    # Itere pelos pontos da linha de ônibus atual.
    for idx, ponto_linha in linha_atual.iterrows():
        ponto_linha_geometry = ponto_linha['geometry']
        # Itere pelas paradas.
        for idx_parada, parada in df_paradas.iterrows():
            parada_geometry = parada['geometry']
            # Calcule a distância entre o ponto da linha de ônibus e a parada em graus decimais.
            distancia = ponto_linha_geometry.distance(parada_geometry)

            # Se a distância for menor que a tolerância, considere a parada parte da rota.
            if distancia < tolerancia:
                
                print(distancia)
                print(parada.y)
                print(parada.x)
                
                # Armazene a parada no dicionário na ordem em que foram comparadas.
                paradas_ordenadas[idx_parada] = {
                    'latitude': parada_geometry.y,
                    'longitude': parada_geometry.x
                }

    # Adicione o dicionário de paradas ordenadas da linha atual ao dicionário geral de todas as paradas.
    todas_as_paradas[num_linha] = paradas_ordenadas

# O dicionário 'todas_as_paradas' agora contém as paradas que fazem parte da rota de todas as linhas de ônibus na ordem de rota.
# Define o nome do arquivo JSON de saída.
output_json_filename = 'paradas_linhas_ida_ordem_teste.json'

# Salva o dicionário em um arquivo JSON.
with open(output_json_filename, 'w') as json_file:
    json.dump(todas_as_paradas, json_file)


/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/luiz/.pyenv/versions/3.11.3/envs/dev/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [10]:
import json

# Carregue o JSON das rotas de linhas.
with open('paradas_linhas_circular.json', 'r') as json_file:
    todas_as_paradas = json.load(json_file)

# Defina o sentido como "ida" para todas as linhas no JSON.
for paradas_na_rota in todas_as_paradas.values():
    for parada_info in paradas_na_rota.values():
        parada_info['sentido'] = 'circular'

# Agora, o JSON 'todas_as_paradas_na_rota_em_ordem.json' foi atualizado com o sentido "ida".
# Você pode salvar o JSON atualizado de volta ao arquivo, se desejar.
with open('paradas_linhas_circular.json', 'w') as json_file:
    json.dump(todas_as_paradas, json_file)

print("Sentido 'ida' definido para todas as linhas e JSON atualizado.")


Sentido 'ida' definido para todas as linhas e JSON atualizado.


In [11]:
import folium
import json

# Substitua 'todas_as_paradas_na_rota_em_ordem.json' pelo caminho do seu arquivo JSON das linhas.
caminho_json_linhas = 'paradas_linhas_ida_ordem_teste.json'

# Substitua '0.205' pela linha específica que você deseja plotar.
linha_especifica = '0.813'

# Carregue o JSON com as informações das linhas.
with open(caminho_json_linhas, 'r') as json_file:
    todas_as_paradas = json.load(json_file)

# Verifique se a linha específica está presente no JSON.
if linha_especifica in todas_as_paradas:
    paradas_na_rota = todas_as_paradas[linha_especifica]

    # Crie um mapa no centro das coordenadas iniciais (por exemplo, a primeira parada da linha específica).
    latitude_inicial = next(iter(paradas_na_rota.values()))['latitude']
    longitude_inicial = next(iter(paradas_na_rota.values()))['longitude']
    m = folium.Map(location=[latitude_inicial, longitude_inicial], zoom_start=15)

    # Crie uma lista de coordenadas para a linha específica.
    coordenadas_linha = [(info['latitude'], info['longitude']) for info in paradas_na_rota.values()]

    # Adicione uma linha polilínea para representar a rota da linha específica.
    #folium.PolyLine(locations=coordenadas_linha, color='blue', weight=3, opacity=0.7).add_to(m)

    # Adicione marcadores para cada parada na rota.
    for parada_id, info in paradas_na_rota.items():
        folium.Marker([info['latitude'], info['longitude']], tooltip=f'Parada {parada_id}').add_to(m)

    # Exiba o mapa.
m


In [12]:
import folium
import json

# Substitua 'todas_as_paradas_na_rota_em_ordem.json' pelo caminho do seu arquivo JSON das linhas.
caminho_json_linhas = 'paradas_linhas_ida_ordem_teste.json'

# Substitua '0.205' pela linha específica que você deseja plotar.
linha_especifica = '0.813'

# Carregue o JSON com as informações das linhas.
with open(caminho_json_linhas, 'r') as json_file:
    todas_as_paradas = json.load(json_file)

# Verifique se a linha específica está presente no JSON.
if linha_especifica in todas_as_paradas:
    paradas_na_rota = todas_as_paradas[linha_especifica]

    # Crie um mapa no centro das coordenadas iniciais (por exemplo, a primeira parada da linha específica).
    latitude_inicial = next(iter(paradas_na_rota.values()))['latitude']
    longitude_inicial = next(iter(paradas_na_rota.values()))['longitude']
    m = folium.Map(location=[latitude_inicial, longitude_inicial], zoom_start=15)

    # Pegue as primeiras 4 paradas da rota.
    primeiras_4_paradas = list(paradas_na_rota.values())[1:10]

    # Adicione marcadores para cada uma das 4 paradas.
    for idx, info in enumerate(primeiras_4_paradas):
        folium.Marker([info['latitude'], info['longitude']], tooltip=f'Parada {idx + 1}').add_to(m)

    # Exiba o mapa.
m


In [15]:
import folium
import json

# Carregue o JSON com as paradas na rota.
with open('paradas_na_rota_final.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Crie um mapa no centro das coordenadas iniciais (por exemplo, a primeira parada).
latitude_inicial = next(iter(paradas_na_rota.values()))['latitude']
longitude_inicial = next(iter(paradas_na_rota.values()))['longitude']
m = folium.Map(location=[latitude_inicial, longitude_inicial], zoom_start=15)

for parada_id, info in paradas_na_rota.items():
    folium.Marker([info['latitude'], info['longitude']], tooltip=f'Parada {parada_id}').add_to(m)

# Exiba o mapa.
m


In [12]:
import folium
import json

# Carregue o JSON com as paradas na rota.
with open('paradas_na_rota_em_ordem.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Crie um mapa no centro das coordenadas iniciais (por exemplo, a primeira parada).
latitude_inicial = next(iter(paradas_na_rota.values()))['latitude']
longitude_inicial = next(iter(paradas_na_rota.values()))['longitude']
m = folium.Map(location=[latitude_inicial, longitude_inicial], zoom_start=15)

# Adicione marcadores para as paradas na rota no mapa.
for parada_id, info in paradas_na_rota.items():
    folium.Marker([info['latitude'], info['longitude']], tooltip=f'Parada {parada_id}').add_to(m)

# Exiba o mapa.
m


In [24]:
import json
from shapely.geometry import Point

# Carregue o arquivo JSON de paradas na rota.
with open('paradas_na_rota_em_ordem.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Converta o dicionário de paradas em uma lista para que possamos iterar por todas.
paradas_na_rota_lista = list(paradas_na_rota.items())

# Defina a tolerância de proximidade em graus decimais.
tolerancia = 0.0009  # Cerca de 10 metros em graus decimais (para EPSG:4326).

# Inicialize uma lista para armazenar as paradas finais.
paradas_finais = [paradas_na_rota_lista[0]]  # Comece com a primeira parada.

# Itere pelas paradas a partir da segunda parada.
for i in range(1, len(paradas_na_rota_lista)):
    parada_atual_idx, parada_atual = paradas_na_rota_lista[i]
    parada_atual_geometry = Point(parada_atual['longitude'], parada_atual['latitude'])
    
    # Verifique a distância entre a parada atual e todas as paradas finais.
    distancia_menor_que_10 = any(parada_atual_geometry.distance(Point(p[1]['longitude'], p[1]['latitude'])) < tolerancia for p in paradas_finais)
    
    if not distancia_menor_que_10:
        paradas_finais.append((parada_atual_idx, parada_atual))

# Converta a lista de paradas finais de volta para um dicionário.
paradas_finais_dict = dict(paradas_finais)

# Atualize o arquivo JSON com as paradas finais.
with open('paradas_na_rota_final.json', 'w') as json_file:
    json.dump(paradas_finais_dict, json_file)

print("Paradas na rota final salvas em 'paradas_na_rota_final.json'.")


Paradas na rota final salvas em 'paradas_na_rota_final.json'.


In [14]:
import json
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
import numpy as np

# Carregue o arquivo JSON de paradas na rota.
with open('paradas_na_rota_em_ordem.json', 'r') as json_file:
    paradas_na_rota = json.load(json_file)

# Extraia as coordenadas (latitude e longitude) das paradas.
coordenadas = [(p['latitude'], p['longitude']) for p in paradas_na_rota.values()]

# Converta as coordenadas em um array NumPy.
X = np.array(coordenadas)

# Defina a distância máxima para considerar paradas como parte do mesmo cluster.
distancia_maxima = 0.0015  # Ajuste conforme necessário.

# Crie um modelo DBSCAN com a distância máxima especificada.
dbscan = DBSCAN(eps=distancia_maxima, min_samples=1)

# Realize o agrupamento das paradas.
labels = dbscan.fit_predict(X)

# Inicialize um dicionário para armazenar as paradas finais.
paradas_finais = {}

# Itere pelas paradas e mantenha apenas uma de cada cluster.
for cluster_id in set(labels):
    # Encontre as paradas pertencentes a este cluster.
    cluster_indices = np.where(labels == cluster_id)[0]
    
    # Escolha uma parada do cluster (por exemplo, a primeira).
    parada_escolhida_idx = cluster_indices[0]
    parada_escolhida = paradas_na_rota[list(paradas_na_rota.keys())[parada_escolhida_idx]]
    
    # Converta o índice para uma string e adicione a parada escolhida ao dicionário de paradas finais.
    paradas_finais[str(parada_escolhida_idx)] = parada_escolhida

# Atualize o arquivo JSON com as paradas finais.
with open('paradas_na_rota_final.json', 'w') as json_file:
    json.dump(paradas_finais, json_file)

print("Paradas na rota final salvas em 'paradas_na_rota_final.json'.")


Paradas na rota final salvas em 'paradas_na_rota_final.json'.


In [152]:
# Converte para o formato correto

def converter_para_EPSG_4326(point):
    from shapely.geometry import Point
    import pyproj

    transformer = pyproj.Transformer.from_crs("epsg:31983", "epsg:4326", always_xy=True)
    
    lon, lat = transformer.transform(point.x, point.y)
    
    ponto_EPSG_4326 = Point(lon, lat)
    
    return ponto_EPSG_4326


In [153]:
from shapely.geometry import Point
from shapely.ops import transform
from functools import partial
import pyproj

def calcular_distancia_km(point1, point2):
    project_to_EPSG_4326 = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:31983'),  # Projeto de origem (EPSG 31983)
        pyproj.Proj(init='epsg:4326'),  # Projeto de destino (EPSG 4326)
    )

    # Transforma os pontos para o EPSG 4326
    point1_wgs84 = transform(project_to_EPSG_4326, point1)
    point2_wgs84 = transform(project_to_EPSG_4326, point2)

    # Calcula a distância entre os pontos em graus
    distancia_graus = point1_wgs84.distance(point2_wgs84)

    # Converte a distância para metros (considerando que 1 grau de latitude ~= 111.32 km)
    distancia_metros = distancia_graus * 111320.0

    return distancia_metros



In [154]:
# Plota gráficos para a gente
import folium

def criar_mapa_com_dois_pontos(coord1, coord2):
    # Cria um objeto de mapa
    m = folium.Map(location=coord1, zoom_start=15)

    # Adiciona um círculo azul para o primeiro ponto
    folium.CircleMarker(
        location=coord1,
        radius=10,  # Tamanho do círculo em pixels
        color='blue',  # Cor do círculo
        fill=True,
        fill_color='blue',  # Cor de preenchimento do círculo
        fill_opacity=0.6,
    ).add_to(m)

    # Adiciona um círculo azul para o segundo ponto
    folium.CircleMarker(
        location=coord2,
        radius=10,  # Tamanho do círculo em pixels
        color='blue',  # Cor do círculo
        fill=True,
        fill_color='blue',  # Cor de preenchimento do círculo
        fill_opacity=0.6,
    ).add_to(m)

    return m

# Exemplo de uso da função
coordenadas1 = [-15.91131954437385, -47.76791017769213]
coordenadas2 = [-15.911596647471995, -47.76792780490262]

mapa = criar_mapa_com_dois_pontos(coordenadas1, coordenadas2)

# Salva o mapa como um arquivo HTML
mapa.save("mapa_com_dois_pontos.html")
m

In [155]:
# Cacular desvio padrão com relação as médias


In [156]:
# Armazenar as distancias.


In [39]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import json
import numpy as np

# Váriaveis onde estão as bases das linhas e paradas
FILE_LINHAS = "../../../databases/linhas/Linhas.shp"
FILE_PARADAS = "../../../databases/paradas/paradas.shp"

# Abrindo e convertendo o dataframe para um formato adquado
df_linhas = gpd.read_file(FILE_LINHAS)
df_paradas = gpd.read_file(FILE_PARADAS)
df_linhas = df_linhas.to_crs(epsg=4326)
df_paradas = df_paradas.to_crs(epsg=4326)

# Filtrando a linha para testar
linha = input("Digite uma linha")
linha_teste = df_linhas[df_linhas['linha'] == linha] 
linestring_rota = linha_teste.iloc[1].geometry.coords
lista_pontos = []
limiar = 0.00019

# Iterando sobre os pontos da linestring da rota e das paradas
for num1, (pontoX_linha, pontoY_linha) in enumerate(linestring_rota):
    for num2, ponto in enumerate(df_paradas.geometry):
        pontoX_parada = ponto.x
        pontoY_parada = ponto.y
        
        # Calculando a distância entre o primeiro ponto da linestring em relação a todos os outros
        distancia = Point(pontoX_linha, pontoY_linha).distance(ponto)
        if distancia < limiar:
            lista_pontos.append(tuple(str(ponto.x), str(ponto.y)))      
        

TypeError: tuple expected at most 1 argument, got 2

In [37]:
lista_pontos

['POINT (-48.10436178761736 -15.923732177932882)',
 'POINT (-48.10301513885338 -15.921286375823692)',
 'POINT (-48.090839 -15.910156000000002)',
 'POINT (-48.07850522929963 -15.905765859448325)',
 'POINT (-48.07138779205818 -15.903887039812487)',
 'POINT (-48.068197398424736 -15.9044180218172)',
 'POINT (-48.068197398424736 -15.9044180218172)',
 'POINT (-48.06538186779065 -15.90429257982713)',
 'POINT (-48.06200257905907 -15.90324937694194)',
 'POINT (-48.04365980236096 -15.874219321801254)',
 'POINT (-48.02948710262496 -15.872196498773105)',
 'POINT (-48.03349066346261 -15.86475268049056)',
 'POINT (-48.04033949037358 -15.851928133271286)',
 'POINT (-48.044656 -15.843727999999999)',
 'POINT (-48.0446454603338 -15.843726586976329)',
 'POINT (-48.045936321652164 -15.8312520086486)',
 'POINT (-48.03584792707868 -15.829871688906936)',
 'POINT (-48.008993671247076 -15.816058132690555)',
 'POINT (-47.99393122588247 -15.810923120761801)',
 'POINT (-47.976921254720494 -15.810939720493455)',
 

In [32]:
import folium

# Seu dicionário de pontos e valores

# Crie um mapa
m = folium.Map(location=[-15.9320432171429, -48.109250444235066], zoom_start=10)

# Adicione marcadores para cada ponto no dicionário
for point_str, value in lista_distancias():
    # Extraia as coordenadas do ponto
    lon, lat = map(float, point_str.replace('POINT (', '').replace(')', '').split())
    
    # Adicione um marcador com uma etiqueta contendo o valor
    folium.Marker([lat, lon], tooltip=str(value)).add_to(m)

# Salve o mapa como um arquivo HTML
m


In [7]:
# Esse script pegar um ponto e a chave corresponde a menor distância entre o primeiro ponto da linestring
min_value = []
for num, chave in enumerate(lista_distancias):
    if num == 0:
        min_value.append(lista_distancias[chave])
        min_value.append(chave)
    else:
        if lista_distancias[chave] < min_value[0]:
            min_value[0] = lista_distancias[chave]
            min_value[1] = chave
        else:
            continue
min_value

[0.001110139777372504, 'POINT (-48.109250444235066 -15.9320432171429)']

In [67]:
# Testantdo para ver se plota a linestring em ordem
import folium
import json

# Adicione marcadores para as paradas na rota no mapa.
for num, (long, lat) in enumerate(a):
    if num == 0:
        m = folium.Map(location=[lat, long], zoom_start=15)
    elif num <= 13:
        folium.Circle([lat, long], radius=10).add_to(m)
    else:
        break
# Exiba o mapa.
m


In [10]:
# Testantdo para ver se plota a linestring em ordem
import folium
import json

lat = -15.9320432171429
long = -48.109250444235066
# Adicione marcadores para as paradas na rota no mapa.
m = folium.Map(location=[lat, long], zoom_start=15)
folium.Circle([lat, long], radius=10).add_to(m)
m
